In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QObject, pyqtSlot
from updatesTest import Ui_MainWindow
import sys
from djitellopy.tello import Tello
import socket
import threading
import pywifi
import time
wifi = pywifi.PyWiFi()

class MainWindowUIClass( Ui_MainWindow ):
    def __init__( self ):
        '''Initialize the super class
        '''
        super().__init__()
        

app = QtWidgets.QApplication(sys.argv)
MainWindow = QtWidgets.QMainWindow()
ui = MainWindowUIClass()
ui.setupUi(MainWindow) 
    
from psutil import process_iter
from signal import SIGTERM # or SIGKILL

for proc in process_iter():
    for conns in proc.connections(kind='inet'):
        if conns.laddr.port == 9090:
            proc.send_signal(SIGTERM)

for proc in process_iter():
    for conns in proc.connections(kind='inet'):
        if conns.laddr.port == 9011:
            proc.send_signal(SIGTERM)

for proc in process_iter():
    for conns in proc.connections(kind='inet'):
        if conns.laddr.port == 9012:
            proc.send_signal(SIGTERM)


# IP and port of Tello
tello_address = ('192.168.10.1', 8889)
tello1_address = ('192.168.0.10', 8889)
tello2_address = ('192.168.0.11', 8889)

# IP and port of local computer
local1_address = ('', 9090)
local2_address = ('', 9011)
local3_address = ('', 9012)

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# sock1 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# sock2 = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock.bind(local1_address)
# sock1.bind(local3_address)
# sock2.bind(local2_address)

def send(message ,delay):
  # Try to send the message otherwise print the exception
    try:
        sock.sendto(message.encode(), tello1_address)
#         sock2.sendto(message.encode(), tello2_address)
        ui.loglist.addItem("Sending message: " + message)
#         print("Sending message: " + message)
    except Exception as e:
        ui.loglist.addItem("Error sending: " + str(e))

  # Delay for a user-defined period of time
    time.sleep(delay)

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
    while True:
    # Try to receive the message otherwise print the exception
        try:
            response1, ip_address = sock.recvfrom(1518)
#         response2, ip_address = sock2.recvfrom(1518)
#         response3, ip_address = sock3.recvfrom(1518)  
            ui.loglist.addItem("Received message: from Tello EDU #1: " + response1.decode(encoding="utf-8"))
#         ui.loglist.addItem("Received message: from Tello EDU #1: " + response2.decode(encoding="utf-8"))
#         ui.loglist.addItem("EDU #1 is not connected")
#         ui.loglist.addItem("Received message: from Tello EDU #2: " + response3.decode(encoding="utf-8"))
#         ui.loglist.addItem("EDU #2 is not connected")
        except Exception as e:
      # If there's an error close the socket and break out of the loop
            sock.close()
#         sock1.close()
#         sock2.close()
            ui.loglist.addItem("Error receiving: " + str(e))
            break

# Create and start a listening thread that runs in the background
# This utilizes our receive functions and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()


       
global tello1
global tello2



def drone1():
    send("command",3)

def drone2():
    send("command",3)
    
def takeoff():
    ui.loglist.addItem("taking off")
    send("takeoff",3)
    
        
def land():
    ui.loglist.addItem("landing")
    send("land",3)
def right():
    ui.loglist.addItem('moving right')
    send("right 40",3)
    
def left():
    ui.loglist.addItem('moving left')
    send("left 40",3)
    
def forward():
    ui.loglist.addItem('moving forward')
    send("forward 40",3)
def back():
    ui.loglist.addItem('moving backward')
    send("back 40",3)

def down():
    ui.loglist.addItem('moving down')
    send("down 40",3)
def up():
    ui.loglist.addItem('moving up')
    send("up 40",3)
def flipRight():
    ui.loglist.addItem('flip right')
    send('flip r',2)
def flipLeft():
    ui.loglist.addItem('flip left')
    send('flip l',2)

def square():
    for i in range(6):
        send("forward 100",0)
  # Delay for a user-defined period of time
        time.sleep(4)
        sendboth("cw 90")
        time.sleep(2)
        
def rectangle():
    for i in range(6):
        send("forward 50",0)
        time.sleep(2)
        sendboth("cw 90",0)
        time.sleep(2)
        sendboth("forward 100",0)
        time.sleep(4)
        sendboth("cw 90",0)
        time.sleep(2)

iface=wifi.interfaces()[1]
def main():
    """
    This is the MAIN ENTRY POINT of our application.  The code at the end
    of the mainwindow.py script will not be executed, since this script is now
    our main program.   We have simply copied the code from mainwindow.py here
    since it was automatically generated by '''pyuic5'''.

    """
    

    
    wifiArr = []
    def Wifi():
        check = 1
        for i in iface.scan_results():
            for j in wifiArr:
                if j == i.ssid:
                    check=0
            if check==1:
                wifiArr.append(i.ssid)
            check=1
        ui.wifidropdown.clear()
        ui.wifidropdown.addItems(wifiArr)
    
    def textChanged():
        ui.deviceName.setText(ui.wifidropdown.currentText())
        if(ui.wifidropdown.currentText()!=''):
            ui.loglist.addItem(ui.wifidropdown.currentText()+' is selected')
            
            
    def connectF():
        ui.loglist.addItem(ui.deviceName.toPlainText()+' is connecting....')
        for i in iface.scan_results():
            if i.ssid==ui.deviceName.toPlainText():
                iface.connect(i)
                time.sleep(2)
                break
        if iface.status()!=4:
            ui.loglist.addItem('unable to connect to a device! please request again.')
        if iface.status()==4:
            ui.loglist.addItem(ui.deviceName.toPlainText()+' is now connected!')
    
    def sendDirect(message ,delay=0):
      # Try to send the message otherwise print the exception
        try:
            sock.sendto(message.encode(), tello_address)
            ui.loglist.addItem("Sending information to drone: " + message)
        except Exception as e:
            ui.loglist.addItem("Error sending: " + str(e))

      # Delay for a user-defined period of time
        time.sleep(delay)
    
    
    def addF():
        rName=ui.routerName.toPlainText()
        rPass=ui.routerPassword.toPlainText()
        message='ap '+rName+' '+rPass
        sendDirect("command")
        time.sleep(0.5)
        sendDirect(message)
        time.sleep(0.5)
        ui.loglist.addItem('Drone is successfully added in the swarm')
                    
        
    ui.refreshBtn.clicked.connect(Wifi)
    ui.wifidropdown.currentIndexChanged.connect(textChanged)
    ui.add.clicked.connect(addF)
    ui.connectBtn.clicked.connect(connectF)
    ui.takeoffbtn.clicked.connect(takeoff)
    ui.landbtn.clicked.connect(land)
    ui.upbtn.clicked.connect(up)
    ui.backbtn.clicked.connect(back)
    ui.downbtn.clicked.connect(down)
    ui.forwardbtn.clicked.connect(forward)
    ui.leftbtn.clicked.connect(left)
    ui.rightbtn.clicked.connect(right)
    ui.flipLeftBtn.clicked.connect(flipLeft)
    ui.flipRightBtn.clicked.connect(flipRight)
#     ui.drone1.clicked.connect(drone1)
#     ui.drone2.clicked.connect(drone2)
    ui.squarebtn.clicked.connect(square)
    ui.rectanglebtn.clicked.connect(rectangle)
    ui.awakeBtn.clicked.connect(drone1)
    
    MainWindow.show()
    sys.exit(app.exec_())
main()

Sending message: command
Sending message: command
Sending message: takeoff
Sending message: takeoff
Sending message: up 40
Sending message: up 40
Sending message: right 40
Sending message: right 40
Sending message: left 40
Sending message: left 40
Sending message: forward 40
Sending message: forward 40
Sending message: down 40
Sending message: down 40
Sending message: land
Sending message: land
Sending message: takeoff
Sending message: takeoff
Sending message: forward 100
Sending message: forward 100


NameError: name 'sendboth' is not defined

Sending message: back 40
Sending message: back 40
Sending message: back 40
Sending message: back 40
Sending message: back 40
Sending message: back 40
Sending message: flip r
Sending message: flip r
Sending message: left 40
Sending message: left 40
Sending message: land
Sending message: land
Sending message: takeoff
Sending message: takeoff
Sending message: up 40
Sending message: up 40
Sending message: down 40
Sending message: down 40
Sending message: forward 40
Sending message: forward 40
Sending message: back 40
Sending message: back 40
Sending message: forward 40
Sending message: forward 40
Sending message: land
Sending message: land
